In [21]:
import pandas as pd
import numpy as np
import warnings

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score
from sklearn.feature_selection import RFE

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [22]:
data = pd.read_csv(path+'data/ml_input.csv')
processed = pd.read_csv(path+'data/processed.csv')
parameters = pd.read_csv(path+'parameters/svm_classifier.csv')

In [23]:
params_to_drop = ['season', 'round', 'driver', 'constructor', 'circuit_id', 'podium', 'driver_points_from']

In [24]:
### Season to test results

N = 2021

In [25]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season < N]

scaler = StandardScaler()
X_train = train.drop(params_to_drop, axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.podium.values)

In [26]:
def score_classification(model, X_train, y_train):
    predictions = []
    prob = []
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.podium

        # Scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # Make Predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns=['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df['predicted'] = prediction_df.proba_1.map(lambda x: 1 if x == prediction_df.proba_1.max() else 0)

        predictions += list(prediction_df['predicted'].values)  
        prob += list(prediction_df['proba_1'].values)

        # Retrain model
        # X_train = pd.concat([X_train, X_test])
        # y_train = np.append(y_train, y_test)

        # model.fit(X_train, y_train)

    return predictions, prob

In [27]:
parameters.head()

,model,gamma,C,kernel,score
0,svm_classifier,0.100000,10.000000,rbf,0.736842
1,svm_classifier,0.100000,6.951928,rbf,0.736842
2,svm_classifier,0.069519,10.000000,rbf,0.736842
3,svm_classifier,0.000428,2.335721,linear,0.684211
4,svm_classifier,0.000298,2.335721,linear,0.684211


In [28]:
params = parameters.iloc[0]

gamma = params.gamma
c = params.C
kernel = params.kernel

params

model     svm_classifier
gamma                0.1
C                   10.0
kernel               rbf
score           0.736842
Name: 0, dtype: object

In [29]:
model = SVC(probability=True, gamma=gamma, C=c, kernel=kernel)
model.fit(X_train, y_train)

predictions, probs = score_classification(model, X_train, y_train)

In [30]:
df = processed.copy()
df = df[(df.season == N)]

df['predicted'] = predictions
df['proba_1'] = probs

In [35]:
df.query('round == 4').sort_values('proba_1', ascending=False)

,season,round,circuit_id,driver,constructor,podium,qualifying_pos,stage,q_delta,starting_grid,...,constructor_points_before,constructor_standings_before,driver_points_from,driver_points_per,constructor_points_per,points_percentage,driver_last_3,constructor_last_3,predicted,proba_1
2824,2021,4,catalunya,lewis_hamilton,mercedes,1,1,q3,0.000,1,...,101.0,1,25.0,17.25,25.25,68.32,69.0,101.0,1,0.307187
2831,2021,4,catalunya,lando_norris,mclaren,8,9,q3,1.269,9,...,53.0,3,4.0,9.25,13.25,69.81,37.0,53.0,0,0.060095
2825,2021,4,catalunya,max_verstappen,red_bull,2,2,q3,0.036,2,...,83.0,2,19.0,15.25,20.75,73.49,61.0,83.0,0,0.052284
2832,2021,4,catalunya,esteban_ocon,alpine,9,5,q3,0.839,5,...,13.0,5,2.0,2.00,3.25,61.54,8.0,13.0,0,0.042288
2842,2021,4,catalunya,nikita_mazepin,haas,19,20,q1,3.066,20,...,0.0,10,0.0,0.00,0.00,0.00,0.0,0.0,0,0.041322
2835,2021,4,catalunya,kimi_raikkonen,alfa,12,17,q1,2.176,17,...,0.0,8,0.0,0.00,0.00,0.00,0.0,0.0,0,0.041261
2838,2021,4,catalunya,antonio_giovinazzi,alfa,15,14,q2,1.615,14,...,0.0,8,0.0,0.00,0.00,0.00,0.0,0.0,0,0.040748
2839,2021,4,catalunya,nicholas_latifi,williams,16,19,q1,2.478,19,...,0.0,9,0.0,0.00,0.00,0.00,0.0,0.0,0,0.040443
2841,2021,4,catalunya,mick_schumacher,haas,18,18,q1,2.376,18,...,0.0,10,0.0,0.00,0.00,0.00,0.0,0.0,0,0.040408
2837,2021,4,catalunya,george_russell,williams,14,15,q2,2.413,15,...,0.0,9,0.0,0.00,0.00,0.00,0.0,0.0,0,0.039970
